In [1]:
# pricing an American put option - Longstaff-Schwarz 
import numpy as np 
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
# parameters of the stock and the option 
S0 = 40.0
K = 40.0 
r = 0.06
sigma = 0.4
T = 2.0

In [3]:
# numeric parameters - number of intervals, number of samples, random numbers etc. 
N = 500
h = T/N 
M = 50000

In [4]:
S = S0*np.ones([M,N+1])
Z = npr.randn(M,N)
b = 3               # number of basis functions - up to 4
disc = np.exp(-r*h)      # one period discount factor 

In [5]:
# generate price paths using the GBM formula
for n in range(0,N): 
    S[:,n+1]=S[:,n]*np.exp((r-0.5*sigma**2)*h + Z[:,n]*sigma*np.sqrt(h))

In [6]:
# exercise value
def exer(SS):
    return((K-SS)*(K>SS))

# Basis functions
def psi(i,SS):
    if i==0:
        return(1)
    elif i==1:
        return((1-SS/K))
    elif i==2:
        return((1-SS/K)**2)
    elif i==3:
        return((1-SS/K)**3)

In [7]:
c = np.zeros([b,N+1])     # coefficients of continuation values
MM = np.zeros([b,b])      # normalization matrix
val = np.zeros([M,N+1])   # option values in simulation 
val[:,N] = exer(S[:,N])   # option values at final time

In [8]:
for k in range(N-1,0,-1):
    sel = S[:,k]<K
    val[~sel,k] =  disc*val[~sel,k+1]
    Ssel = S[sel,k]
    valsel = disc*val[sel,k+1]
    for i in range(0,b):
        c[i,k] = np.mean( valsel *  psi(i,Ssel))      # was called d in file 
        MM[i,i] = np.mean( psi(i,Ssel) *  psi(i,Ssel) )   
        for j in range(i+1,b):
            MM[i,j] = np.mean( psi(i,Ssel) *  psi(j,Ssel) ) 
            MM[j,i] = MM[i,j] 
    c[:,k] = np.linalg.solve(MM,c[:,k])
    cont = np.zeros(Ssel.size)
    for i in range(0,b):
        cont = cont + c[i,k] * psi(i,Ssel)
    val[sel,k] = exer(Ssel) * (exer(Ssel) > cont) + disc*val[sel,k+1] * (exer(Ssel) < cont)
val[:,0] =  disc*val[:,1]
V1=np.mean(val[:,0])
e1=np.std(val[:,0])/np.sqrt(M)
print("price of the option is: {:.3f}, with a stochastic error of {:.5f}".format(V1,e1))

price of the option is: 6.948, with a stochastic error of 0.03037


In [9]:
# generate a further set of paths to get lower bound on option price 
S0 = 40
M2 = 100000
S2 = S0*np.ones([M2,N+1])
Z2 = npr.randn(M2,N)
for n in range(0,N): 
    tem =(r-0.5*sigma**2)*h + np.sqrt(h)*sigma*Z2[:,n] 
    S2[:,n+1]=S2[:,n]*np.exp(tem)
val2 = np.zeros(M2)
for n in range(1,N+1): 
    tem1 = exer( S2[:,n] )
    tem2 = np.zeros(M2)
    for i in range(0,b):
        tem2 = tem2 + c[i,n] * psi(i,S2[:,n])
    val2 = val2 + np.exp(-r*n*h)*(val2==0)*(tem1>tem2)*tem1
V2=np.mean(val2)
e2=np.std(val2)/np.sqrt(M2)
print("price of the option is: {:.3f}, with a stochastic error of {:.5f}".format(V2,e2))    

price of the option is: 6.878, with a stochastic error of 0.02144
